In [19]:
import pandas as pd

In [18]:
basics = pd.read_csv("../data/title.basics.tsv.gz", sep='\t', dtype=str, na_values='\\N')


In [20]:
basics.shape

(11639217, 9)

In [24]:
# Find avengers endgame movie row
avengers_endgame = basics[(basics['primaryTitle'] == 'Avengers: Endgame') & (basics['titleType'] == 'movie')]
avengers_endgame

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9043478,tt4154796,movie,Avengers: Endgame,Avengers: Endgame,0,2019,NaN,181,"Action,Adventure,Sci-Fi"


In [ ]:
ratings = pd.read_csv("../data/title.ratings.tsv.gz", sep='\t', dtype=str, na_values='\\N')



(1565473, 3)

In [28]:
# Find avengers endgame ratings row
avengers_endgame_ratings = ratings[ratings['tconst'] == avengers_endgame['tconst'].values[0]]
avengers_endgame_ratings

,tconst,averageRating,numVotes
1254161,tt4154796,8.4,1354985


In [29]:
akas = pd.read_csv("../data/title.akas.tsv.gz", sep='\t', dtype=str, na_values='\\N')


In [32]:
# Find avengers endgame akas row
avengers_endgame_akas = akas[akas['titleId'] == avengers_endgame['tconst'].values[0]]
avengers_endgame_akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
40857240,tt4154796,1,Avengers: Endgame,NaN,NaN,original,NaN,1
40857241,tt4154796,10,Avengers: Endgame,CZ,NaN,imdbDisplay,NaN,0
40857242,tt4154796,11,Avengers: Endgame,DE,NaN,imdbDisplay,NaN,0
40857243,tt4154796,12,Avengers: Endgame,EC,NaN,imdbDisplay,NaN,0
40857244,tt4154796,13,Avengers: Endgame,EG,en,imdbDisplay,NaN,0
...,...,...,...,...,...,...,...,...
40857310,tt4154796,73,复联4,CN,cmn,alternative,NaN,0
40857311,tt4154796,74,復仇者聯盟：終局之戰,TW,NaN,imdbDisplay,NaN,0
40857312,tt4154796,75,Avengers: Endgame,US,en,NaN,NaN,0
40857313,tt4154796,8,Avengers: Endgame,CL,NaN,imdbDisplay,NaN,0


In [ ]:

# Load data
basics = pd.read_csv("../data/title.basics.tsv.gz", sep='\t', dtype=str, na_values='\\N')
ratings = pd.read_csv("../data/title.ratings.tsv.gz", sep='\t', dtype=str, na_values='\\N')
akas = pd.read_csv("../data/title.akas.tsv.gz", sep='\t', dtype=str, na_values='\\N')

# Filter to only movies
basics = basics[basics['titleType'] == 'movie']

# Merge title.basics with ratings
df = basics.merge(ratings, on='tconst', how='left')

# Aggregate production countries from akas (all available regions)
region_df = akas[['titleId', 'region']].dropna()
region_agg = region_df.groupby('titleId')['region'].agg(lambda x: '|'.join(sorted(set(x)))).reset_index()
region_agg = region_agg.rename(columns={'titleId': 'tconst', 'region': 'production_country'})

# Merge country info
df = df.merge(region_agg, on='tconst', how='left')

# Select and rename columns
df_final = df[[
    'tconst', 'primaryTitle', 'originalTitle', 'startYear',
    'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'production_country'
]].rename(columns={
    'tconst': 'title_id',
    'primaryTitle': 'title',
    'originalTitle': 'original_title',
    'startYear': 'release_year',
    'runtimeMinutes': 'runtime_minutes',
    'genres': 'genre',
    'averageRating': 'imdb_rating',
    'numVotes': 'vote_count'
})

# Convert types
df_final['release_year'] = pd.to_numeric(df_final['release_year'], errors='coerce')
df_final['runtime_minutes'] = pd.to_numeric(df_final['runtime_minutes'], errors='coerce')
df_final['imdb_rating'] = pd.to_numeric(df_final['imdb_rating'], errors='coerce')
df_final['vote_count'] = pd.to_numeric(df_final['vote_count'], errors='coerce')

# Verify columns
print(df_final.columns)
print(df_final.head())


Index(['title_id', 'title', 'original_title', 'release_year',
       'runtime_minutes', 'genre', 'imdb_rating', 'vote_count',
       'production_country'],
      dtype='object')
    title_id                          title                 original_title  \
0  tt0000009                     Miss Jerry                     Miss Jerry   
1  tt0000147  The Corbett-Fitzsimmons Fight  The Corbett-Fitzsimmons Fight   
2  tt0000502                       Bohemios                       Bohemios   
3  tt0000574    The Story of the Kelly Gang    The Story of the Kelly Gang   
4  tt0000591               The Prodigal Son              L'enfant prodigue   

   release_year  runtime_minutes                       genre  imdb_rating  \
0        1894.0             45.0                     Romance          5.4   
1        1897.0            100.0      Documentary,News,Sport          5.3   
2        1905.0            100.0                         NaN          3.8   
3        1906.0             70.0  Action,Adve

In [22]:
df_final.shape

(714015, 9)

In [15]:
# Convert year and minutes to integers
df_final['release_year'] = df_final['release_year'].astype('Int64')
df_final['runtime_minutes'] = df_final['runtime_minutes'].astype('Int64')

# Convert vote_count to integer
df_final['vote_count'] = df_final['vote_count'].astype('Int64')


In [16]:
df_final.head()

,title_id,title,original_title,release_year,runtime_minutes,genre,imdb_rating,vote_count,production_country
0,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.4,223,AU|DE|HU|US
1,tt0000147,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,1897,100,"Documentary,News,Sport",5.3,559,RU|US
2,tt0000502,Bohemios,Bohemios,1905,100,NaN,3.8,21,ES
3,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography",6.0,985,AU|DE|GB|HU|RS|SG|US
4,tt0000591,The Prodigal Son,L'enfant prodigue,1907,90,Drama,5.6,31,FR|US


In [25]:
# save as parquet../data/movies.parquet
df_final.to_parquet("../data/movies.parquet", index=False, engine='pyarrow')

In [26]:
df_final.shape

(714015, 9)

In [17]:
def describe_dataframe_for_markdown(df):
    summary = []
    for col in df.columns:
        dtype = df[col].dtype
        non_null = df[col].notnull().sum()
        nulls = df[col].isnull().sum()
        unique = df[col].nunique(dropna=True)
        example = df[col].dropna().iloc[0] if df[col].notna().any() else "NaN"
        summary.append([col, str(dtype), non_null, nulls, unique, str(example)])

    header = "| Column | Data Type | Non-Missing | Missing | Unique Values | Example |"
    divider = "|--------|------------|--------------|---------|----------------|---------|"
    lines = [header, divider]
    for row in summary:
        lines.append("| " + " | ".join(map(str, row)) + " |")
    
    markdown_output = "\n".join(lines)
    print(markdown_output)

# Use it on your df_final
describe_dataframe_for_markdown(df_final)


| Column | Data Type | Non-Missing | Missing | Unique Values | Example |
|--------|------------|--------------|---------|----------------|---------|
| title_id | object | 714015 | 0 | 714015 | tt0000009 |
| title | object | 714013 | 2 | 613714 | Miss Jerry |
| original_title | object | 714013 | 2 | 627750 | Miss Jerry |
| release_year | Int64 | 608940 | 105075 | 139 | 1894 |
| runtime_minutes | Int64 | 450303 | 263712 | 510 | 45 |
| genre | object | 637026 | 76989 | 1494 | Romance |
| imdb_rating | float64 | 329613 | 384402 | 91 | 5.4 |
| vote_count | Int64 | 329613 | 384402 | 19061 | 223 |
| production_country | object | 699897 | 14118 | 118177 | AU|DE|HU|US |
